In [ ]:
# 1. global alignment
# Insert your GlobalAlignment function here, along with any subroutines you need
def GlobalAlignment(s,t, match_reward, mismatch_penalty, indel_penalty) :
    # it is a t*s matrix
    ScoreMatrix = [[0 for _ in range(len(s)+1)]for _ in range(len(t)+1)]
    backtrack = [[None for _ in range(len(s)+1)]for _ in range(len(t)+1)]
    # base case
    # for the first row, it should be an addition of indel penalty
    for i in range(1, len(s)+1):
        ScoreMatrix[0][i] = ScoreMatrix[0][i-1] - indel_penalty
        backtrack[0][i] = "t_gap"
    # for the first column
    for j in range(1,len(t)+1):
        ScoreMatrix[j][0] = ScoreMatrix[j-1][0] - indel_penalty
        backtrack[j][0] = "s_gap"
    
    # traverse every other state, here index = nucleotide_position + 1 
    for i in range(1, len(s)+1): # i denotes the column index actually
        for j in range(1,len(t)+1):
            # the diagnoal value including match or mismatch condition
            diagonal = 0
            if s[i-1] == t[j-1]:
                diagonal = ScoreMatrix[j-1][i-1]+match_reward
            else:
                diagonal = ScoreMatrix[j-1][i-1]-mismatch_penalty
            
            ScoreMatrix[j][i] = max(diagonal, ScoreMatrix[j-1][i]-indel_penalty, ScoreMatrix[j][i-1]-indel_penalty)
            
            if ScoreMatrix[j][i] == diagonal:
                backtrack[j][i] = "diagonal"
            elif ScoreMatrix[j][i] == ScoreMatrix[j-1][i] - indel_penalty:
                backtrack[j][i] = "s_gap" # the down arrow
            else:
                backtrack[j][i] = "t_gap" # the right arrow
      
    final_score = ScoreMatrix[len(t)][len(s)]
    s_aligned, t_aligned = BackTrack(s,t, backtrack)
    
    return final_score, s_aligned, t_aligned

# BackTrack is a recursion finding the alignment path, but it is to inefficient 
# def BackTrack(s, t, i, j, backtrack):
#     # base case
#     # at sink
#     if i == 0 and j == 0:
#         return "",""
#     # in the first column
#     elif i == 0 and j != 0:
#         return BackTrack(s,t,i,j-1, backtrack)[0]+"-", BackTrack(s,t,i,j-1, backtrack)[1]+t[j-1]
#     # in the first row
#     elif i != 0 and j == 0:
#         return BackTrack(s,t,i-1,j, backtrack)[0]+s[i-1], BackTrack(s,t,i-1,j, backtrack)[1]+"-"
    
#     # common case
#     if backtrack[j][i] == "diagonal":
#         return BackTrack(s,t,i-1,j-1, backtrack)[0] + s[i-1], BackTrack(s,t,i-1,j-1, backtrack)[1] + t[j-1]
#     if backtrack[j][i] == "s_gap":
#         return BackTrack(s,t,i,j-1, backtrack)[0]+"-", BackTrack(s,t,i,j-1, backtrack)[1] + t[j-1]
#     if backtrack[j][i] == "t_gap":
#         return BackTrack(s,t,i-1,j, backtrack)[0]+s[i-1], BackTrack(s,t,i-1,j, backtrack)[1] + "-"
        
# an iterative backtrack function
def BackTrack(s, t, backtrack):
    i, j = len(s), len(t)
    s_aligned, t_aligned = "", ""

    # iteratively traverse the backtrack matrix
    while i > 0 or j > 0:
        if i > 0 and j > 0 and backtrack[j][i] == "diagonal":
            s_aligned = s[i - 1] + s_aligned
            t_aligned = t[j - 1] + t_aligned
            i -= 1
            j -= 1
        elif j > 0 and backtrack[j][i] == "s_gap":
            s_aligned = "-" + s_aligned
            t_aligned = t[j - 1] + t_aligned
            j -= 1
        else:  # t_gap or reaching the edge of the matrix
            s_aligned = s[i - 1] + s_aligned
            t_aligned = "-" + t_aligned
            i -= 1

    return s_aligned, t_aligned

# print(GlobalAlignment(s,t,match_reward, mismatch_penalty, indel_penalty))


# 2. Local alignment
def LocalAlignment(match_reward: int, mismatch_penalty: int, indel_penalty: int,
                    s: str, t: str) -> Tuple[int, str, str]:
    # it is a t*s matrix
    ScoreMatrix = [[0 for _ in range(len(s)+1)]for _ in range(len(t)+1)]
    backtrack = [[None for _ in range(len(s)+1)]for _ in range(len(t)+1)]
    # in local alignment, we backtrack from the max score cell rather than the last cell
    # the max_score stores the max score, its index row and column
    max_score = [0,0,0]
    
    # base case
    # for the first row, it should be an addition of indel penalty
    # but include the free ride, it should be max(0,any score)
    for i in range(1, len(s)+1):
        ScoreMatrix[0][i] = 0
        backtrack[0][i] = "new_start"
    # for the first column
    for j in range(1,len(t)+1):
        ScoreMatrix[j][0] = 0
        backtrack[j][0] = "new_start"
    
    # traverse every other state, here index = nucleotide_position + 1 
    for i in range(1, len(s)+1): # i denotes the column index actually
        for j in range(1,len(t)+1):
            # the diagnoal value including match or mismatch condition
            diagonal = 0
            if s[i-1] == t[j-1]:
                diagonal = ScoreMatrix[j-1][i-1]+match_reward
            else:
                diagonal = ScoreMatrix[j-1][i-1]-mismatch_penalty
            
            ScoreMatrix[j][i] = max(0, diagonal, ScoreMatrix[j-1][i]-indel_penalty, ScoreMatrix[j][i-1]-indel_penalty)
            
            # update the max score so far
            if ScoreMatrix[j][i] > max_score[0]:
                max_score[0] = ScoreMatrix[j][i]
                max_score[1] = j
                max_score[2] = i
            
            if ScoreMatrix[j][i] == diagonal:
                backtrack[j][i] = "diagonal"
            elif ScoreMatrix[j][i] == ScoreMatrix[j-1][i] - indel_penalty:
                backtrack[j][i] = "s_gap" # the down arrow
            elif ScoreMatrix[j][i] == 0:
                backtrack[j][i] = "new_start" # the local alignment sink
            else:
                backtrack[j][i] = "t_gap" # the right arrow
      
    final_score = max_score[0]
    s_aligned, t_aligned = BackTrack(s, t, max_score, backtrack)
    
    return final_score, s_aligned, t_aligned

# # BackTrack is a recursion finding the alignment path
def BackTrack(s, t, max_score, backtrack):
    j, i = max_score[1], max_score[2]
    s_aligned, t_aligned = "", ""

    # iteratively traverse the backtrack matrix
    while i > 0 or j > 0:
        if i > 0 and j > 0 and backtrack[j][i] == "diagonal":
            s_aligned = s[i - 1] + s_aligned
            t_aligned = t[j - 1] + t_aligned
            i -= 1
            j -= 1
        elif j > 0 and backtrack[j][i] == "s_gap":
            s_aligned = "-" + s_aligned
            t_aligned = t[j - 1] + t_aligned
            j -= 1
        elif i > 0 and backtrack[j][i] == "t_gap":  # t_gap or reaching the edge of the matrix
            s_aligned = s[i - 1] + s_aligned
            t_aligned = "-" + t_aligned
            i -= 1 
        else: # meet "new_start"
            break
    return s_aligned, t_aligned


# 3. edit distance
def EditDistance(s: str, t: str) -> int:
    match_reward = 0
    indel_penalty = 1
    mismatch_penalty = 1
    # it is a t*s matrix
    ScoreMatrix = [[0 for _ in range(len(s)+1)]for _ in range(len(t)+1)]
    # base case
    # for the first row, it should be an addition of indel penalty
    for i in range(1, len(s)+1):
        ScoreMatrix[0][i] = ScoreMatrix[0][i-1] - indel_penalty

    # for the first column
    for j in range(1,len(t)+1):
        ScoreMatrix[j][0] = ScoreMatrix[j-1][0] - indel_penalty
  
    
    # traverse every other state, here index = nucleotide_position + 1 
    for i in range(1, len(s)+1): # i denotes the column index actually
        for j in range(1,len(t)+1):
            # the diagnoal value including match or mismatch condition
            diagonal = 0
            if s[i-1] == t[j-1]:
                diagonal = ScoreMatrix[j-1][i-1]+match_reward
            else:
                diagonal = ScoreMatrix[j-1][i-1]-mismatch_penalty
            
            ScoreMatrix[j][i] = max(diagonal, ScoreMatrix[j-1][i]-indel_penalty, ScoreMatrix[j][i-1]-indel_penalty)
            

    final_score = 0 - ScoreMatrix[len(t)][len(s)]
    
    
    return final_score

# 4. Fitting alignment
def FittingAlignment(s: str, t: str,
                     BLOSUM: Dict[str, Dict[str, int]], indel_penalty) -> Tuple[int, str, str]:
    # it is a t*s matrix, s is the long string, t is the short one
    ScoreMatrix = [[0 for _ in range(len(s)+1)]for _ in range(len(t)+1)]
    backtrack = [[None for _ in range(len(s)+1)]for _ in range(len(t)+1)]
    
    
    # base case
    # for the first row, it should be an addition of indel penalty
    # but include the free ride, it should be max(0,any score),  free ride in s axis
    for i in range(1, len(s)+1):
        ScoreMatrix[0][i] = 0
        backtrack[0][i] = "new_start"
    # for the first column
    for j in range(1,len(t)+1):
        ScoreMatrix[j][0] = ScoreMatrix[j][0]-indel_penalty
        backtrack[j][0] = "s_gap"
    
    # traverse every other state, here index = nucleotide_position + 1 
    for i in range(1, len(s)+1): # i denotes the column index actually
        for j in range(1,len(t)+1):
            # the diagnoal value including match or mismatch condition
            diagonal = ScoreMatrix[j-1][i-1]+BLOSUM[s[i-1]][t[j-1]]
            
            ScoreMatrix[j][i] = max(diagonal, ScoreMatrix[j-1][i]-indel_penalty, ScoreMatrix[j][i-1]-indel_penalty)
            # new start is only allowed at the start of t
            
            if ScoreMatrix[j][i] == diagonal:
                backtrack[j][i] = "diagonal"
            elif ScoreMatrix[j][i] == ScoreMatrix[j-1][i] - indel_penalty:
                backtrack[j][i] = "s_gap" # the down arrow
            else:
                backtrack[j][i] = "t_gap" # the right arrow
                
    last_row = ScoreMatrix[-1]
    max_index, max_score = max(enumerate(last_row), key=lambda x: x[1])
  
    final_score = max_score
    s_aligned, t_aligned = BackTrack(s, t, max_index, backtrack)

    return final_score, s_aligned, t_aligned

# # BackTrack is a recursion finding the alignment path
def BackTrack(s, t, max_index, backtrack):
    j, i = len(t), max_index
    s_aligned, t_aligned = "", ""

    # iteratively traverse the backtrack matrix
    while i > 0 or j > 0:
        if i > 0 and j > 0 and backtrack[j][i] == "diagonal":
            s_aligned = s[i - 1] + s_aligned
            t_aligned = t[j - 1] + t_aligned
            i -= 1
            j -= 1
        elif j > 0 and backtrack[j][i] == "s_gap":
            s_aligned = "-" + s_aligned
            t_aligned = t[j - 1] + t_aligned
            j -= 1
        elif i > 0 and backtrack[j][i] == "t_gap":  # t_gap or reaching the edge of the matrix
            s_aligned = s[i - 1] + s_aligned
            t_aligned = "-" + t_aligned
            i -= 1 
        else: # meet "new_start"
            break
    return s_aligned, t_aligned

# 5. Overlap alignment
def OverlapAlignment(match_reward: int, mismatch_penalty: int, indel_penalty: int,
                    s: str, t: str) -> Tuple[int, str, str]:
    # it is a t*s matrix, s is the long string, t is the short one
    ScoreMatrix = [[0 for _ in range(len(s)+1)]for _ in range(len(t)+1)]
    backtrack = [[None for _ in range(len(s)+1)]for _ in range(len(t)+1)]
    
    
    # base case
    # for the first row, it should be an addition of indel penalty
    # but include the free ride, it should be max(0,any score),  free ride in s axis
    for i in range(1, len(s)+1):
        ScoreMatrix[0][i] = 0
        backtrack[0][i] = "new_start"
    # for the first column
    for j in range(1,len(t)+1):
        ScoreMatrix[j][0] = ScoreMatrix[j-1][0]-indel_penalty
        backtrack[j][0] = "s_gap"
    
    # traverse every other state, here index = nucleotide_position + 1 
    for i in range(1, len(s)+1): # i denotes the column index actually
        for j in range(1,len(t)+1):
            # the diagnoal value including match or mismatch condition
            if s[i-1] == t[j-1]:
                diagonal = ScoreMatrix[j-1][i-1]+ match_reward
            else:
                diagonal = ScoreMatrix[j-1][i-1] - mismatch_penalty
            
            ScoreMatrix[j][i] = max(diagonal, ScoreMatrix[j-1][i]-indel_penalty, ScoreMatrix[j][i-1]-indel_penalty)
            
            
            if ScoreMatrix[j][i] == diagonal:
                backtrack[j][i] = "diagonal"
            elif ScoreMatrix[j][i] == ScoreMatrix[j-1][i] - indel_penalty:
                backtrack[j][i] = "s_gap" # the down arrow
            else:
                backtrack[j][i] = "t_gap" # the right arrow
                
    last_col = [ScoreMatrix[i][len(s)] for i in range(len(t)+1)]
    max_index, max_score = max(enumerate(last_col), key=lambda x: x[1])
  
    final_score = max_score
    s_aligned, t_aligned = BackTrack(s, t, max_index, backtrack)

    return final_score, s_aligned, t_aligned

# BackTrack is a recursion finding the alignment path
def BackTrack(s, t, max_index, backtrack):
    j, i = max_index, len(s)
    s_aligned, t_aligned = "", ""

    # iteratively traverse the backtrack matrix
    while i > 0 or j > 0:
        if i > 0 and j > 0 and backtrack[j][i] == "diagonal":
            s_aligned = s[i - 1] + s_aligned
            t_aligned = t[j - 1] + t_aligned
            i -= 1
            j -= 1
        elif j > 0 and backtrack[j][i] == "s_gap":
            s_aligned = "-" + s_aligned
            t_aligned = t[j - 1] + t_aligned
            j -= 1
        elif i > 0 and backtrack[j][i] == "t_gap":  # t_gap or reaching the edge of the matrix
            s_aligned = s[i - 1] + s_aligned
            t_aligned = "-" + t_aligned
            i -= 1 
        else: # meet "new_start"
            break
    return s_aligned, t_aligned

# 6. Affine Alignment
def AffineAlignment(match_reward: int, mismatch_penalty: int,
                    gap_opening_penalty: int, gap_extension_penalty: int,
                    s: str, t: str) :
    # it is a s*t matrix
    middle = [[0 for _ in range(len(t)+1)]for _ in range(len(s)+1)]
    # down arrow, s_gap
    lower = [[float('-inf') for _ in range(len(t)+1)]for _ in range(len(s)+1)]
    # right, t_gap
    upper = [[float('-inf') for _ in range(len(t)+1)]for _ in range(len(s)+1)]
    
    # base case
    # initialize the first column and first row of middle, which will initilize the corresponding
    # column and row of lower and upper in the next big for loops because of original value of "-inf"
    for i in range(1, len(s) + 1):
        middle[i][0] = - gap_opening_penalty - (i-1) * gap_extension_penalty 
        
    for j in range(1, len(t) + 1):
        middle[0][j] = - gap_opening_penalty - (j-1) * gap_extension_penalty
        
    
    
    # traverse every other state, here index = nucleotide_position + 1 
    for i in range(1, len(s)+1): # i denotes the row index now
        for j in range(1,len(t)+1):
            upper[i][j] = max(upper[i][j-1]-gap_extension_penalty, middle[i][j-1]-gap_opening_penalty)
            lower[i][j] = max(lower[i-1][j]-gap_extension_penalty, middle[i-1][j]-gap_opening_penalty)
   
            # the diagnoal value including match or mismatch condition
            if s[i-1] == t[j-1]:
                diagonal = middle[i-1][j-1] + match_reward
            else :
                diagonal = middle[i-1][j-1] - mismatch_penalty
            # notice that it is possible that two or three of these three values are equal
            # we need to decide priorities of match/mismatch and continuous gaps in backtrack process
            middle[i][j] = max(diagonal, lower[i][j], upper[i][j])
            
    final_score = middle[len(s)][len(t)]
    
    i, j = len(s), len(t)
    s_aligned, t_aligned = "", ""

    # iteratively traverse the backtrack matrix
    while i > 0 and j > 0:
        # since match rewards, it is with higher priority
        if middle[i][j] == middle[i-1][j-1]+(match_reward if s[i - 1] == t[j - 1] else -mismatch_penalty):
            s_aligned = s[i - 1] + s_aligned
            t_aligned = t[j - 1] + t_aligned
            i -= 1
            j -= 1
        elif middle[i][j] == upper[i][j]:
            # this decribe continuous s gap, we will assign higher priority on this
            while j > 0 and middle[i][j] == upper[i][j]:
                s_aligned = "-" + s_aligned
                t_aligned = t[j - 1] + t_aligned
                j -= 1
        elif middle[i][j] == lower[i][j]:  # t_gap or reaching the edge of the matrix
            while i > 0 and middle[i][j] == lower[i][j]:
                s_aligned = s[i - 1] + s_aligned
                t_aligned = "-" + t_aligned
                i -= 1
    
    # considering the case of j = 0 or i = 0
    while i > 0:
        s_aligned = s[i - 1] + s_aligned
        t_aligned = "-" + t_aligned
        i -= 1
        
    while j > 0:
        s_aligned = "-" + s_aligned
        t_aligned = t[j - 1] + t_aligned
        j -= 1

    return final_score, s_aligned, t_aligned


# 7. MultipleAlign
def MultipleAlignment(s1: str, s2: str, s3: str) -> Tuple[int, str, str, str]:
    
    scoreMatrix = [[[0 for _ in range(len(s3)+1)] for _ in range(len(s2)+1)] for _ in range(len(s1)+1)]
    
    for i in range(1, len(s1)+1):
        for j in range(1, len(s2)+1):
            for k in range(1, len(s3)+1):
                # if match
                if s1[i-1] == s2[j-1] == s3[k-1]:
                    scoreMatrix[i][j][k] = scoreMatrix[i-1][j-1][k-1]+1
                else:
                    scoreMatrix[i][j][k] = max(scoreMatrix[i-1][j][k], # s1,-,-
                                              scoreMatrix[i][j-1][k], # -,s2,-
                                              scoreMatrix[i][j][k-1], # -,-,s3
                                              scoreMatrix[i-1][j-1][k], # s1,s2,-
                                              scoreMatrix[i-1][j][k-1], # s1,-,s3
                                              scoreMatrix[i][j-1][k-1]) # -,s2,s3
                
    
   # backtrack the scoreMatrix, we only consider in each column it is either match or only 1 element and 2 gaps
    s1_align, s2_align, s3_align = "","",""
    i,j,k = len(s1),len(s2),len(s3)
    while i > 0 and j > 0 and k > 0:
        if s1[i-1] == s2[j-1] == s3[k-1] and scoreMatrix[i][j][k] == scoreMatrix[i-1][j-1][k-1]+1:
            s1_align = s1[i-1] + s1_align
            s2_align = s2[j-1] + s2_align
            s3_align = s3[k-1] + s3_align
            i -= 1
            j -= 1
            k -= 1
        elif scoreMatrix[i][j][k] == scoreMatrix[i-1][j][k]:
            s1_align = s1[i-1] + s1_align
            s2_align = "-" + s2_align
            s3_align = "-" + s3_align
            i -= 1
        elif scoreMatrix[i][j][k] == scoreMatrix[i][j-1][k]:
            s1_align = "-" + s1_align
            s2_align = s2[j-1] + s2_align
            s3_align = "-" + s3_align
            j -= 1
        elif scoreMatrix[i][j][k] == scoreMatrix[i][j][k-1]:
            s1_align = "-" + s1_align
            s2_align = "-" + s2_align
            s3_align = s3[k-1] + s3_align
            k -= 1
     
    # any one of the three index are decreased to 0, then we decrease another 2 to 0 respectively
    while i > 0:
        s1_align = s1[i-1] + s1_align
        s2_align = "-" + s2_align
        s3_align = "-" + s3_align
        i -= 1
   
    while j > 0:
        s1_align = "-" + s1_align
        s2_align = s2[j-1] + s2_align
        s3_align = "-" + s3_align
        j -= 1
     
    while k > 0:
        s1_align = "-" + s1_align
        s2_align = "-" + s2_align
        s3_align = s3[k-1] + s3_align
        k -= 1

    return scoreMatrix[len(s1)][len(s2)][len(s3)], s1_align, s2_align, s3_align